<a href="https://colab.research.google.com/github/tazar09/situatii_financiare_2022/blob/main/sit_fin_2023%5B04-dec2023%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [117]:
import pandas as pd
import numpy as np
import re
import tabula
pd.set_option('display.max_rows', 250)
import warnings
warnings.filterwarnings("ignore")

# Import coduri pentru impozite

# New Section

In [2]:
#Import cod impozit dataframe
url_cod = '/content/cod_impozit.csv'
lista_impozite = pd.read_csv(url_cod)
lista_impozite['COD_IMP'] = 'IMP' + lista_impozite['COD_IMP'].astype(str)
lista_impozite.head()

,COD_IMP,SEMNIFICATIE,COD_IMP_FK,DATA_INCEPUT
0,IMP100,Profit,100.0,01.01.2000
1,IMP101,Profit BNR,100.0,01.01.2000
2,IMP102,Profit banci,100.0,01.01.2000
3,IMP103,Profit agenti economici,100.0,01.01.2000
4,IMP104,Profit anual,100.0,01.01.2000


# Import situatii financiare anul 2022

In [3]:
#Import situatii financiare dataframe
url_web_bl_bs_2022 = 'https://data.gov.ro/dataset/aa2567a4-e7d7-4e6e-ab19-d08d39f99996/resource/b35fab04-f101-42d7-a765-8f41728b373a/download/web_bl_bs_sl_an2022.txt'
df_base = pd.read_csv(url_web_bl_bs_2022, dtype = {'CUI':str, 'CAEN':str}, thousands = ',')

In [4]:
df = df_base.copy()
df.shape

(76639, 22)

In [6]:
df.head()

,CUI,CAEN,I1,I2,I3,I4,I5,I6,I7,I8,...,I11,I12,I13,I14,I15,I16,I17,I18,I19,I20
0,1758080,4931,16250707.0,12600287.0,922470.0,11449101.0,228716.0,650906.0,27949623,21426.0,...,210755.0,NaN,39213937.0,40326671.0,39905042.0,421629.0,0.0,274227.0,0.0,NaN
1,1770555,2899,4382308.0,13167071.0,6243032.0,3050034.0,3874005.0,25919.0,7307533,NaN,...,720.0,NaN,21480601.0,21616592.0,18740693.0,2875899.0,0.0,2486912.0,0.0,99.0
2,1772076,1431,3991824.0,13670457.0,9949344.0,2795638.0,925475.0,35908.0,6032140,486167.0,...,3990125.0,NaN,19048965.0,21765244.0,21733706.0,31538.0,0.0,31538.0,0.0,197.0
3,1774336,4941,21747409.0,12896215.0,10201046.0,1945071.0,750098.0,450.0,1258049,NaN,...,15000.0,NaN,16695296.0,13086584.0,11190142.0,1896442.0,0.0,1645894.0,0.0,19.0
4,2763904,1089,3186213.0,53223368.0,36399030.0,6481835.0,10342503.0,15708.0,23711439,NaN,...,17000000.0,NaN,42440258.0,43968685.0,40528519.0,3440166.0,0.0,3035703.0,0.0,46.0


# Formatare coloane main dataframe

## Import df coloane.

In [7]:
#Import coloane dataframe
url_coloane_2022 = 'https://data.gov.ro/dataset/aa2567a4-e7d7-4e6e-ab19-d08d39f99996/resource/0079a6b0-0a5a-4be9-a26a-857449f41c25/download/web_bl_bs_sl_an2022.csv'
coloane = pd.read_csv(url_coloane_2022)

## Redenumire coloane.

In [8]:
list_ind = list(coloane['CUI;CUI'])
list_ind.insert(0, 'CUI')
len(list_ind)

22

In [9]:
df.columns = list_ind

In [10]:
columns_names = list(df.columns)

In [11]:
# Regex pattern
regex = r';i\d{1,2}$'

# Apply the regex pattern to the column names
new_column_names = []
for column_name in columns_names:
    new_column_name = re.sub(regex, '', column_name)
    new_column_names.append(new_column_name)

In [12]:
df.columns = new_column_names
df.columns = df.columns.str.lower()

In [13]:
df.columns = ['cui', 'caen', 'activ_total',
       'activ_circ', 'stocuri', 'creante',
       'casa_banci', 'chel_avans', 'datorii',
       'ven_avans', 'proviz', 'KP',
       'cap_subs_varsat', 'patrim_reg',
       'CA', 'ven_tot', 'chelt_tot',
       'prof_brut', 'pierd_bruta', 'prof_net', 'pierd_neta',
       'salar_med']

In [14]:
df.shape

(76639, 22)

In [15]:
df.head()

,cui,caen,activ_total,activ_circ,stocuri,creante,casa_banci,chel_avans,datorii,ven_avans,...,cap_subs_varsat,patrim_reg,CA,ven_tot,chelt_tot,prof_brut,pierd_bruta,prof_net,pierd_neta,salar_med
0,1758080,4931,16250707.0,12600287.0,922470.0,11449101.0,228716.0,650906.0,27949623,21426.0,...,210755.0,NaN,39213937.0,40326671.0,39905042.0,421629.0,0.0,274227.0,0.0,NaN
1,1770555,2899,4382308.0,13167071.0,6243032.0,3050034.0,3874005.0,25919.0,7307533,NaN,...,720.0,NaN,21480601.0,21616592.0,18740693.0,2875899.0,0.0,2486912.0,0.0,99.0
2,1772076,1431,3991824.0,13670457.0,9949344.0,2795638.0,925475.0,35908.0,6032140,486167.0,...,3990125.0,NaN,19048965.0,21765244.0,21733706.0,31538.0,0.0,31538.0,0.0,197.0
3,1774336,4941,21747409.0,12896215.0,10201046.0,1945071.0,750098.0,450.0,1258049,NaN,...,15000.0,NaN,16695296.0,13086584.0,11190142.0,1896442.0,0.0,1645894.0,0.0,19.0
4,2763904,1089,3186213.0,53223368.0,36399030.0,6481835.0,10342503.0,15708.0,23711439,NaN,...,17000000.0,NaN,42440258.0,43968685.0,40528519.0,3440166.0,0.0,3035703.0,0.0,46.0


# Import lista de platitori

In [86]:
#Import lista platitori
url_platitori = '/content/date_identificar_platitori_2023_txt..txt'
dic = {'COD_FISCAL': str, 'DENUMIRE': str}

In [87]:
lista_platitori_base = pd.read_csv(url_platitori, sep = '|', header=0, usecols= ['COD_FISCAL', 'DENUMIRE'], low_memory=False, index_col = None, dtype= dic)

In [88]:
lista_platitori = lista_platitori_base.copy()
lista_platitori.shape

(2457148, 2)

In [89]:
lista_platitori = lista_platitori.reset_index(names = 'cod_fisc')

def format_scientific_to_string(value):
  return "{:.0f}".format(value)

lista_platitori['cod_fisc'] = lista_platitori['cod_fisc'].apply(lambda x: format_scientific_to_string(x))

In [90]:
lista_platitori = lista_platitori.reset_index(drop = True)

In [91]:
lista_platitori.columns = ['cui', 'denumire', 'x']
lista_platitori.drop('x', axis = 1, inplace = True)

In [ ]:
lista_platitori.head()

In [93]:
lista_platitori[lista_platitori['denumire'].str.contains('huawei', case=False, na=False)]

,cui,denumire
1209820,37175141,HUAWEI TECH INVESTMENT CO.LTD.
1361754,39908320,HUAWEI TECHNOLOGIES CO LTD
1385143,40350285,HUAWEI ELECTRONICS ROMANIA S.R.L.
2057153,20567140,HUAWEI TECHNOLOGIES SRL
2256577,37168888,HUAWEI INTERNATIONAL PTE LTD


In [94]:
df['cui'] = df['cui'].str.strip()
lista_platitori['cui'] = lista_platitori['cui'].str.strip()

In [95]:
df1 = df.merge(lista_platitori, on='cui', how= 'left')
df1['denumire'].isna().sum()

1217

In [ ]:
df1[['cui', 'caen', 'denumire']].head()

In [98]:
df1[['cui', 'caen', 'denumire']].shape

(76639, 3)

In [99]:
df1[df1['denumire'].str.contains('huawei', case=False, na=False)].denumire

7833    HUAWEI TECHNOLOGIES SRL
Name: denumire, dtype: object

In [105]:
df[df['cui'].str.contains('40350285', case=False, na=False)]

,cui,caen,activ_total,activ_circ,stocuri,creante,casa_banci,chel_avans,datorii,ven_avans,...,cap_subs_varsat,patrim_reg,CA,ven_tot,chelt_tot,prof_brut,pierd_bruta,prof_net,pierd_neta,salar_med


In [107]:
df1.columns

Index(['cui', 'caen', 'activ_total', 'activ_circ', 'stocuri', 'creante',
       'casa_banci', 'chel_avans', 'datorii', 'ven_avans', 'proviz', 'KP',
       'cap_subs_varsat', 'patrim_reg', 'CA', 'ven_tot', 'chelt_tot',
       'prof_brut', 'pierd_bruta', 'prof_net', 'pierd_neta', 'salar_med',
       'denumire'],
      dtype='object')

In [108]:
df1 = df1[['cui', 'denumire','caen', 'CA', 'prof_brut', 'pierd_bruta', 'prof_net', 'pierd_neta']]

In [109]:
df1.sort_values(by = 'CA', ascending = False).head(20)

,cui,denumire,caen,CA,prof_brut,pierd_bruta,prof_net,pierd_neta
5794,11201891,OMV PETROM MARKETING SRL,4730,2.704728e+10,6.469903e+08,0.0,5.558400e+08,0.0
5230,160796,AUTOMOBILE-DACIA SA,2910,2.570914e+10,6.504855e+08,0.0,5.267838e+08,0.0
76446,22891860,LIDL DISCOUNT SRL,4711,1.849600e+10,1.265317e+09,0.0,1.054581e+09,0.0
6292,12751583,ROMPETROL DOWNSTREAM SRL,4730,1.657221e+10,2.268885e+08,0.0,1.900139e+08,0.0
76217,15991149,KAUFLAND ROMANIA SCS,4711,1.601443e+10,1.061400e+09,0.0,8.843465e+08,0.0
6512,22043010,E.ON ENERGIE ROMÂNIA SA,3523,1.372537e+10,0.000000e+00,358899374.0,0.000000e+00,358899374.0
5586,10547022,LUKOIL ROMANIA SRL,4730,1.335891e+10,2.788114e+08,0.0,2.350836e+08,0.0
4983,8808452,BRITISH AMERICAN TOBACCO (ROMANIA) TRADING SRL,4635,1.270139e+10,1.348915e+08,0.0,9.158537e+07,0.0
5980,11607939,PROFI ROM FOOD SRL,4711,1.166066e+10,0.000000e+00,257775507.0,0.000000e+00,285968097.0
7748,28909028,ELECTRICA FURNIZARE SA,3514,1.123515e+10,3.714864e+08,0.0,3.706203e+08,0.0


#Lista celor 1117 societati cu CA peste 50 mil. Euro (sursa: Economedia)

In [112]:
!pip install --upgrade tabula-py
from tabula.io import read_pdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 23.5 MB/s eta 0:00:00


In [118]:
pdf_path = 'https://cdn.economedia.ro/wp-content/uploads/2023/10/Companii-afaceri-peste-50-milioane-euro.pdf'
tables = tabula.read_pdf(pdf_path, pages='all', multiple_tables=False)

Dec 04, 2023 7:24:47 PM org.apache.pdfbox.contentstream.operator.text.SetFontAndSize process
Dec 04, 2023 7:24:47 PM org.apache.pdfbox.contentstream.operator.text.SetFontAndSize process
Dec 04, 2023 7:24:47 PM org.apache.pdfbox.contentstream.operator.text.SetFontAndSize process
Dec 04, 2023 7:24:47 PM org.apache.pdfbox.contentstream.operator.text.SetFontAndSize process
Dec 04, 2023 7:24:47 PM org.apache.pdfbox.contentstream.operator.text.SetFontAndSize process
Dec 04, 2023 7:24:48 PM org.apache.pdfbox.contentstream.operator.text.SetFontAndSize process
Dec 04, 2023 7:24:48 PM org.apache.pdfbox.contentstream.operator.text.SetFontAndSize process
Dec 04, 2023 7:24:48 PM org.apache.pdfbox.contentstream.operator.text.SetFontAndSize process
Dec 04, 2023 7:24:48 PM org.apache.pdfbox.contentstream.operator.text.SetFontAndSize process
Dec 04, 2023 7:24:48 PM org.apache.pdfbox.contentstream.operator.text.SetFontAndSize process
Dec 04, 2023 7:24:48 PM org.apache.pdfbox.contentstream.operator.text.

In [120]:
df = tables[0]
df.head(10)

,Firma,Cifra de afaceri\r(Lei),Profit net\r(Lei),P i e r deri nete\r(Lei),N r . mediu\rangajați
0,OMV PETROM SA,55.939.915.254,10.287.553.182,0,7.372
1,OMV PETROM MARKETING SRL,27.047.276.075,555.840.030,0,187.000
2,AUTOMOBILE-DACIA SA,25.709.139.352,526.783.840,0,12.195
3,ROMPETROL RAFINARE SA,23.026.777.606,666.277.159,0,1.074
4,LIDL DISCOUNT SRL,18.495.999.406,1.054.580.568,0,10.086
5,ROMPETROL DOWNSTREAM SRL,16.572.205.378,190.013.893,0,438.000
6,KAUFLAND ROMANIA SCS,16.014.434.265,884.346.491,0,13.827
7,ENGIE ROMANIA S.A.,15.190.517.642,637.666.691,0,830.000
8,E.ON ENERGIE ROMÂNIA SA,13.725.371.052,0,358.899.374,668.000
9,LUKOIL ROMANIA SRL,13.358.908.234,235.083.572,0,896.000


In [121]:
df.shape

(1117, 5)

In [ ]:
# lista_platitori.columns = ['COD_FISCAL', 'DENUMIRE', 'COD_FISCAL_PARINTE', 'TIP_UNITATE',
#        'TIP_CONTRIB', 'LOCALITATE', 'STRADA', 'NR', 'DATA_INREGISTRARE',
#        'DATA_PRELUCRARE', 'FAX', 'SECTOR', 'TELEFON', 'JUDET_COMERT',
#        'NR_COMERT', 'AN_COMERT', 'ACT_AUTORIZARE', 'TVA', 'DATA_RADIERE',
#        'COD_POSTAL', 'DATA_STARE', 'STARE', 'JUDET', 'IMP100', 'IMP120',
#        'IMP130', 'IMP160', 'ACCIZE200', 'CONT410', 'CONT412', 'CONT416',
#        'CONT420', 'CONT423', 'CONT430', 'CONT432', 'IMP439', 'IMP451',
#        'IMP461', 'IMP473', 'IMP474', 'CONT480', 'IMP500', 'IMP602', 'IMP611',
#        'IMP625', 'IMP632', 'IMP633', 'IMP634', 'IMP701', 'IMP710', 'IMP711',
#        'IMP712', 'IMP713', 'IMP755', 'IMP756', 'IMP940', 'DETALII_ADRESA',
#        'BLOC', 'SCARA', 'ETAJ', 'AP', 'empty']

In [ ]:
# lista_platitori.columns = lista_platitori.columns.str.lower()
# lista_platitori['cod_fiscal'] = lista_platitori['cod_fiscal'].astype(str)

## Selectie coloane din lista platitori.

In [ ]:
# lista_platitori_short = lista_platitori[['cod_fiscal', 'denumire', 'imp100']]
# lista_platitori_short.head(5)

In [ ]:
# lista_platitori_short.shape

## Formatare coloane as str.

In [ ]:
# lista_platitori_short = lista_platitori_short.dropna(axis = 0)
# lista_platitori_short['cod_fiscal'] = lista_platitori_short['cod_fiscal'].astype(float)
# lista_platitori_short['cod_fiscal'] = lista_platitori_short['cod_fiscal'].astype(int)
# lista_platitori_short['cod_fiscal'] = lista_platitori_short['cod_fiscal'].astype(str)
# lista_platitori_short.dtypes
# lista_platitori_short.isna().sum()
# lista_platitori_short = lista_platitori_short[lista_platitori_short['imp100'].str.contains('DA')]
# lista_platitori_short.rename(columns = {'cod_fiscal': 'cui'}, inplace = True)
# lista_platitori_short.head(2)

# Merging to main dataframe

In [ ]:
#Merging
# df = df.merge(lista_platitori_short, on= 'cui', how = 'left')

In [ ]:
# df= df[['cui', 'denumire','caen', 'activ_total', 'activ_circ', 'stocuri', 'creante',
#        'casa_banci', 'chel_avans', 'datorii', 'ven_avans', 'proviz', 'KP',
#        'cap_subs_varsat', 'patrim_reg', 'CA', 'ven_tot', 'chelt_tot',
#        'prof_brut', 'pierd_bruta', 'prof_net', 'pierd_neta', 'salar_med']]

# Exporting the main dataframe

In [ ]:
# df.to_csv('platitori_profit_2022.csv', index=False)

# EDA

In [ ]:
# import pandas as pd
# pd.options.display.max_rows = 999

In [ ]:
# df = pd.read_csv('/content/platitori_profit_2022.csv')


In [ ]:
# df['denumire'] = df['denumire'].str.replace('S.R.L.', '')
# df['denumire'] = df['denumire'].str.replace('SRL', '')
# df['denumire'] = df['denumire'].str.replace('SA', '')
# df['denumire'] = df['denumire'].str.replace('S.A.', '')

In [ ]:
# df.shape

In [ ]:
# df.dropna(subset = 'denumire', axis = 0, inplace = True)

In [ ]:
# df[df['denumire'].isna()].shape

In [ ]:
# publice = ['COMPANIA NAŢIONALĂ', 'REGIA', 'TRANSPORT PUBLIC', 'DRUMURI SI PODURI', 'GOSPODARIREA COMUNALA', 'OCOLUL SILVIC', 'AEROPORT', 'PARCUL INDUSTRIAL',
#            'DOMENIU PUBLIC', 'DOMENIUL PUBLIC','PUBLICĂ URBANĂ', 'A PATRIMONIULUI', 'UTILITĂŢI PUBLICE', 'DE SERVICII PUBLICE', 'ADMINISTRAŢIA DOMENIULUI PUBLIC',
#            'SERVICII DE UTILITATE PUBLICĂ',
#            'TRANSPORT URBAN PUBLIC', 'ADMINISTRARE A DOMENIULUI', 'SERVICII PUBLICE DE', 'GOSPODĂRIA COMUNALĂ',	'UTILITATI PUBLICE',
#            'SERVICII PUBLICE', 'ADMINISTRAREA DOMENIULUI',
#            'DIRECŢIA PUBLICĂ', 'SERVICIUL PUBLIC LOCAL', 'SERVICII ADMINISTRATIV-PUBLICE', 'SOCIETATEA DE ADMINISTRARE A DOMENIULUI',
#            'SERVICIUL PUBLIC DE ALIMENTARE CU APA', 'APĂ CANAL',
#            'COMPANIA MUNICIPALĂ', 'PAZĂ PUBLICĂ',' PUBLIC DE ALIMENTARE', 'TECUCI', 'ALIMENTATIE PUBLICA', 'DRUMURI', 'APA SERV TOP STÂLPENI', 'SERVICII EDILITARE BĂLILEŞTI',
#            'EDILUL APĂ CETĂŢENI', 'SERVICII MUNICIPALE', 'COMUNALĂ', 'AGENŢIA DE', 'TRANSPORT LOCAL', 'SERVICII LOCALE', 'GOSPODĂRIRE', 'GOSPODĂRIE', 'GOSPODARIRE',
#            'COMUNALA', 'GOSPODĂRIA', 'SERVICIUL GOSPODĂRESC', 'SERVICII COMUNALE', 'COMUNALE', 'ROMSILVA', 'SERVICII EDILITARE', 'SERVICII COMUNITARE',
#            'PIEŢE', 'APĂ-CANAL', 'APĂ ŞI CANAL', 'ALIMENTARE CU APĂ', 'APA CANAL', 'APA-CANAL', 'SOCIETATEA DE']
# public_pattern = '|'.join(publice)
# public_pattern

In [ ]:
# df[df['denumire'].str.contains(public_pattern)].shape
# df = df[~df['denumire'].str.contains(public_pattern)]
# df[df['denumire'].str.contains(public_pattern)]
# df[df['denumire'].str.contains('COMUNAL')].shape
# df[df['denumire'].str.contains('SOCIETATEA DE')]

In [ ]:
# df10 = df[df['CA'] > 1000]
# df10.head()
# pd.set_option('display.max_rows', 500)
# df_short = df[['cui','denumire','caen', 'CA', 'pierd_bruta', 'pierd_neta', 'prof_brut', 'prof_net']]
# mask = df_short['pierd_neta'] > 0
# df_short[mask].head(10)
# df_short['rentab_net'] = df_short['prof_net'] - df_short['pierd_neta']
# df_short['rentab_net_proc'] = round(df_short['rentab_net']/df_short['CA'],3)
# df_short.shape[0]
# mask_rentab = (df_short['rentab_net_proc'] > 0) & (df_short['rentab_net_proc'] < 0.1)
# df_short = df_short[mask_rentab]
# df_short.sort_values('rentab_net_proc', ascending = False)
# df_short['rentab_net_proc'].plot(kind = 'hist', bins = 50)

In [ ]:
# # column_mask = ['CUI', 'Cifra de afaceri neta;i13', 'DATORII;i7','Profitul brut;i16','Pierdere bruta;i17', 'Profitul net;i18', 'Pierdere neta;i19']
# df.shape